# Project 3 - Physics Informed Neural Networks

#### Notebook initial imports and defaults

In [1]:
import autograd.numpy as anp
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from autograd import grad, elementwise_grad
from sklearn.metrics import r2_score